In [1]:
using Plots

In [2]:
import LowLevelFEM as FEM
using LowLevelFEM
using SparseArrays

gmsh.initialize()

In [ ]:
#gmsh.open("wave_in_rectangle.geo")
gmsh.open("wave_in_rectangle_2.geo")

In [ ]:
mat = FEM.material("body", E=2.e5, ν=0.3, ρ=7.85e-9)
problem = FEM.Problem([mat], type="PlaneStress")

In [5]:
supp1 = FEM.displacementConstraint("left", ux=0)
supp2 = FEM.displacementConstraint("middle", uy=0)
load = FEM.load("source", fx=-1);

In [6]:
dof = problem.non * problem.dim
u0 = zeros(dof)
v0 = zeros(dof);

In [ ]:
K = FEM.stiffnessMatrix(problem)
M = FEM.massMatrix(problem)
f = FEM.loadVector(problem, [load])
C = spzeros(dof, dof)
dof

In [8]:
FEM.applyBoundaryConditions!(problem, K, M, C, f, [supp1, supp2]);

In [ ]:
Tₘᵢₙ = FEM.smallestPeriodTime(K, M)

In [ ]:
u1, v1, t1 = FEM.CDM(K, M, C, f, u0, v0, Tₘᵢₙ, Tₘᵢₙ / π)

In [ ]:
sts = length(t1)
u0 = u1[:, sts]
v0 = v1[:, sts]

f .*= 0
u2, v2, t2 = FEM.CDM(K, M, C, f, u0, v0, 20Tₘᵢₙ, Tₘᵢₙ / π)

In [ ]:
sts = length(t1)
u0 = u1[:, sts]
v0 = v1[:, sts]

f .*= 0
u3, v3, t3 = FEM.HHT(K, M, f, u0, v0, 20Tₘᵢₙ, Tₘᵢₙ / π)

In [ ]:
plt = FEM.HHTdampingCharacter(2π / Tₘᵢₙ, 10Tₘᵢₙ / π, γ=0.9, β=0.49)
plot(plt)

In [14]:
gmsh.option.setNumber("General.Trackball", 0)
gmsh.option.setNumber("General.RotationX", 298)
gmsh.option.setNumber("General.RotationY", 346)
gmsh.option.setNumber("General.RotationZ", 332)

In [ ]:
vvec1 = FEM.showDoFResults(problem, v1, t=t1, "vvec", name="v1(t)", visible=false)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$vvec1].NormalRaise", 5)
gmsh.option.setNumber("View[$vvec1].VectorType", 5)
gmsh.option.setNumber("View[$vvec1].IntervalsType", 3)
gmsh.option.setNumber("View[$vvec1].NbIso", 20)

In [ ]:
vvec2 = FEM.showDoFResults(problem, v2, t=t2, "vvec", name="v2(t) CDM", visible=true)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$vvec2].NormalRaise", 5)
gmsh.option.setNumber("View[$vvec2].VectorType", 5)
gmsh.option.setNumber("View[$vvec2].IntervalsType", 3)
gmsh.option.setNumber("View[$vvec2].NbIso", 20)

In [ ]:
vvec3 = FEM.showDoFResults(problem, v3, t=t3, "vvec", name="v3(t) HHT", visible=false)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$vvec3].NormalRaise", 5)
gmsh.option.setNumber("View[$vvec3].VectorType", 5)
gmsh.option.setNumber("View[$vvec3].IntervalsType", 3)
gmsh.option.setNumber("View[$vvec3].NbIso", 20)

In [ ]:
S2 = FEM.solveStress(problem, u2)

In [ ]:
s2 = FEM.showStressResults(problem, S2, "seqv", name="σ(t)", t=t2, visible=false, smooth=true)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("View[$s2].NormalRaise", 100)
#gmsh.option.setNumber("View[$s2].VectorType", 5)
gmsh.option.setNumber("View[$s2].IntervalsType", 3)
gmsh.option.setNumber("View[$s2].NbIso", 20)

In [20]:
#seqv = FEM.showStressResults(problem, S2, "seqv", name="σeqv(t)", t=t2, visible=false, smooth=true)
#gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
#gmsh.option.setNumber("View[$seqv].NormalRaise", 100)
##gmsh.option.setNumber("View[$seqv].VectorType", 5)
#gmsh.option.setNumber("View[$seqv].IntervalsType", 3)
#gmsh.option.setNumber("View[$seqv].NbIso", 20)

In [ ]:
ph, x, y = FEM.plotOnPath(problem, "path", s2, 500, step=100, plot=true)
#ph = FEM.plotOnPath(problem, "path", s2, 500, step=1, plot=false)

In [ ]:
plot(x, y)

In [ ]:
gmsh.fltk.run()

In [ ]:
dataType, tags, data, time, numComponents = gmsh.view.getModelData(s2, 0)

In [25]:
gmsh.finalize()

In [ ]:
function vec_test(a; b=false)
    if b == true
        b = [i for i in 1:a]
        return a, b
    else
        return a
    end
end

vec_test(3, b=true)

In [ ]:
using Plots
x = 0:0.01:2π
y = sin.(x)
z = x, y
plot(z)

In [ ]:
range(0, length=10, stop=10)

In [ ]:
findmax([1, 2, 5, 3])
maximum(abs, [1, -2 + 8im, 5im, 3])

In [30]:
using LinearAlgebra
A = [10 30im 0; 30 90 0; 0 0 20]
eig = eigen(A)
maxval, idx = findmax(abs, eig.values)
#eig.values[idx]